In [1]:
!gpustat

TnT                       Fri Sep 29 22:21:02 2023  515.86.01
[0] NVIDIA A100-PCIE-40GB | 56°C,  99 % |  5599 / 40960 MB | code_sheep(947M) hss0729(1973M) hss0729(2109M)
[1] NVIDIA A100-PCIE-40GB | 58°C, 100 % | 10749 / 40960 MB | hss0729(6251M) hss0729(2077M) hss0729(1853M)
[2] NVIDIA A100-PCIE-40GB | 41°C,   0 % |  8484 / 40960 MB | ruili(3959M) ruili(3959M)
[3] NVIDIA A100-PCIE-40GB | 73°C,  96 % | 29355 / 40960 MB | zhiding(28787M)
[4] NVIDIA A100-PCIE-40GB | 66°C,  89 % | 37825 / 40960 MB | yhzhang(37259M)
[5] NVIDIA A100-PCIE-40GB | 68°C,  96 % | 37827 / 40960 MB | yhzhang(37259M)
[6] NVIDIA A100-PCIE-40GB | 63°C,  95 % | 20787 / 40960 MB | code_sheep(20219M)
[7] NVIDIA A100-PCIE-40GB | 25°C,   0 % |   566 / 40960 MB |


In [2]:
import sys
import json
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

import fire
# import gradio as gr
import torch
import transformers
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer, AutoTokenizer, AutoModelForCausalLM, AutoModel, TextStreamer

# from utils.prompter import Prompter
import bitsandbytes as bnb
import warnings
warnings.filterwarnings('ignore')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /data/Yucong/anaconda3/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /data/Yucong/anaconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /data/Yucong/anaconda3/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/data/Yucong/anaconda3/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/data/Yucong/anaconda3/lib/libcudart.so.11.0'), PosixPath('/data/Yucong/anaconda3/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
2023-09-15 11:07:48.510777: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
print(torch.cuda.is_available())

True


In [4]:
load_8bit = False
base_model = 'meta-llama/Llama-2-7b-chat-hf'
model_path = 'meta-llama/Llama-2-7b-chat-hf'
model_name = 'LLaMA2'

if torch.cuda.is_available():
    device = "cuda"

In [5]:
if model_name in ['ziya', 'Qwen', 'LLaMA2']:
    print('Loading tokenizer...')
    tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

    print('Loading model...')
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=load_8bit,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )

    if not load_8bit:
        model.half()  # seems to fix bugs for some users.

    model.eval()

    if torch.__version__ >= "2" and sys.platform != "win32":
        model = torch.compile(model)


Loading tokenizer...
Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
import json
import os.path as osp
from typing import Union


class Prompter_LLaMA2(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, verbose: bool = False):
        super().__init__()

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
    ) -> str:        
        if input:
            prompt = instruction + input
        else:
            prompt = instruction
        system_message = "Please give a proper response to the instruction. Do not say 'I don't know."
        prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]'''
        
        return prompt_template

    def get_response(self, output: str) -> str:
        return output.split('[/INST]')[-1].strip(tokenizer.eos_token).strip()

prompter = Prompter_LLaMA2()

def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=1,
    max_new_tokens=4096,
    **kwargs,
):
    
    streamer = TextStreamer(tokenizer)
    prompt = prompter.generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
            # streamer=streamer,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s, skip_special_tokens=True)
    return instruction, prompter.get_response(output)

In [7]:
instruction = 'hello'
instruction, response = evaluate(instruction)
print(response)

Hello! It's nice to meet you. How can I assist you today?


In [ ]:
import pandas as pd
from recbole.quick_start import load_data_and_model
dataset_name = 'steam'
rec_model_path = '../../RecBole/experiments/steam/BPR/BPR-Aug-28-2023_11-58-38.pth'

config, rec_model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file=rec_model_path,
)

df_item = pd.read_csv(f'dataset/{dataset_name}/{dataset_name}.item', sep='\t')
df_inter = pd.read_csv(f'dataset/{dataset_name}/{dataset_name}.inter', sep='\t')
if 'ml' in dataset_name:
    df_user = pd.read_csv(f'dataset/{dataset_name}/{dataset_name}.user', sep='\t')

In [9]:
from recbole.utils.case_study import full_sort_topk
def get_instruction_ml(idx):
    # user_info
    user_info = df_user[df_user["user_id:token"] == int(idx)]
    user_age = user_info["age:token"].values[0]
    user_gender = 'male' if user_info["gender:token"].values[0] == 'M' else 'female'
    user_occupation = user_info["occupation:token"].values[0]

    # history_inter_info
    df_inter_user = df_inter[df_inter['user_id:token'] == int(idx)]
    his_id_list = df_inter_user['item_id:token'].tolist()[-50:] 
    uid_series = dataset.token2id(dataset.uid_field, [idx])
    topk_score, topk_iid_list = full_sort_topk(uid_series, rec_model, test_data, k=1, device=config['device'])
    rec_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu()) 

    # next_item_info
    next_item = df_item[df_item["item_id:token"] == int(rec_list)]
    next_item_title = next_item["movie_title:token_seq"].values[0]
    next_item_class = next_item["class:token_seq"].values[0]

    instruction = 'The history films watched by the customer are:\n'
    for i, id in enumerate(his_id_list):
        instruction += f'{i+1}: {df_item[df_item["item_id:token"] == id]["movie_title:token_seq"].values[0]}' + '\n'
        instruction += f'The class of the movie is {df_item[df_item["item_id:token"] == id]["class:token_seq"].values[0]}.\n'

    # print(f"The age of the customer is {user_age}, the gender is {user_gender} and the customer's occupation is {user_occupation}.")


    instruction += f"""
As a recommender system in the movie domain, based on the customer's historical viewing records, \
historical viewing movie information and user information, \
tell the customer why he or she needs to watch this movie, and what are the advantages of this movie.
Give reasons why the customer needs to watch this movie with the following title and class:
{next_item_title}.
The class of the movie is {next_item_class}.

Use ONLY the information mentioned above, especially the history record. \
You're responding to the CUSTOMER. Directly tell the customer the reason. \
Respond in SHORT and CONCISE language. Your response should be WITHIN 75 words. \
You can infer the customer's information from the history record, such as age, gender, occupation, etc.

The output format is as follows:
user information: <age, gender, occupation>
recommendation reason: <reason>

"""
    return instruction

def get_instruction_Amazon(idx):
    # history_inter_info
    df_inter_user = df_inter[df_inter['user_id:token'] == idx]
    his_id_list = df_inter_user['item_id:token'].tolist()[-10:]
    uid_series = dataset.token2id(dataset.uid_field, [idx])
    topk_score, topk_iid_list = full_sort_topk(uid_series, rec_model, test_data, k=1, device=config['device'])
    rec_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu()) 
    print(rec_list)
    # next_item_info
    next_item = df_item[df_item["item_id:token"] == rec_list]
    next_item_title = next_item["title:token"].values[0]
    next_item_brand = next_item["brand:token"].values[0]
    brand_prompt = f'The brand of the movie is {next_item_brand}.\n' if isinstance(next_item_brand, str) else ''
    next_item_type = next_item["sales_type:token"].values[0]
    type_prompt = f'The type of the product is {next_item_type}.\n' if isinstance(next_item_type, str) else ''

    instruction = 'The history products bought by the customer are:\n'
    print(his_id_list)
    for i, id in enumerate(his_id_list):
        item = df_item[df_item["item_id:token"] == id]
        item_title = item["title:token"].values[0]
        item_brand = item["brand:token"].values[0]
        item_type = item["sales_type:token"].values[0]

        instruction += f'{i+1}: {item_title}' + '\n'
        instruction += f'The brand of the movie is {item_brand}.\n' if math.isnan(item_brand) else ''
        instruction += f'The type of the product is {item_type}.\n' if math.isnan(item_type) else ''

    instruction += f"""
The age of the customer is {user_age}, the gender is {user_gender} and the customer's occupation is {user_occupation}. \
As a recommender system in the movie domain, based on the customer's HISTORICAL viewing records, \
HISTORICAL viewing movie information and user information, \
tell the customer why he or she needs to watch this movie, and what are the advantages of this movie.
Give reasons why the customer needs to watch this movie with the following title and class:
{next_item_title}.
{brand_prompt}
{type_prompt}

Use ONLY the information mentioned above, especially the history record. \
You're responding to the CUSTOMER. Directly tell the customer the reason. \
Respond in SHORT and CONCISE language. Your response should be WITHIN 75 words.

"""
    return instruction

def get_instruction_mind(idx):
    # history_inter_info
    df_inter_user = df_inter[df_inter['user_id:token'] == int(idx)]
    his_id_list = df_inter_user['item_id:token'].tolist()[-10:]
    uid_series = dataset.token2id(dataset.uid_field, [idx])
    topk_score, topk_iid_list = full_sort_topk(uid_series, rec_model, test_data, k=1, device=config['device'])
    rec_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu()) 
    # next_item_info
    next_item = df_item[df_item["item_id:token"] == int(rec_list)]
    next_item_title = next_item["title:token_seq"].values[0]
    next_item_category = next_item["category:token_seq"].values[0]
    next_item_sub_category = next_item["sub_category:token_seq"].values[0]
    next_item_abstract = next_item["abstract:token_seq"].values[0]


    instruction = 'The history news viewed by the customer are:\n'
    for i, id in enumerate(his_id_list):
        item = df_item[df_item["item_id:token"] == id]
        item_title = item["title:token_seq"].values[0]
        item_category = item["category:token_seq"].values[0]
        item_sub_category = item["sub_category:token_seq"].values[0]

        instruction += f'{i+1}: {item_title}' + '\n'
        instruction += f'The category of the news is {item_category}.\n'
        instruction += f'The sub category of the news is {item_sub_category}.\n'

    instruction += f"""
As a recommender system in the news domin, according to the user's previous news viewing history , \
tell the customer why he or she needs to watch this news.
Give reasons why the customer needs to watch this news with the following title and category:
{next_item_title}.
The category of the news is {next_item_category}.
The sub category of the news is {next_item_sub_category}.

Use ONLY the information mentioned above, especially the history record. \
You're responding to the CUSTOMER. Directly tell the customer the reason. \
Respond in SHORT and CONCISE language. Your response should be WITHIN 75 words.

"""
    return instruction

def get_instruction_steam(idx):
    # history_inter_info
    df_inter_user = df_inter[df_inter['user_id:token'] == int(idx)]
    his_id_list = df_inter_user['product_id:token'].tolist()[-10:]
    uid_series = dataset.token2id(dataset.uid_field, [idx])
    topk_score, topk_iid_list = full_sort_topk(uid_series, rec_model, test_data, k=1, device=config['device'])
    rec_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu()) 
    # next_item_info
    next_item = df_item[df_item["id:token"] == int(rec_list)]
    next_item_title = next_item["app_name:token"].values[0]
    next_item_tag = next_item["genres:token_seq"].values[0]
    next_item_price = next_item["price:float"].values[0]


    instruction = 'The history games played by the customer are:\n'
    for i, id in enumerate(his_id_list):
        item = df_item[df_item["id:token"] == id]
        item_title = item["app_name:token"].values[0]
        item_tag = item["genres:token_seq"].values[0]

        instruction += f'{i+1}: {item_title}' + '\n'
        instruction += f'The tags of the game are {item_tag}.\n'

    instruction += f"""
As a recommender system in the game playing domain, according to the user's HISTORICAL play record , \
tell the customer why he or she needs to play this game, and what are the advantages of this game.
Give reasons why the customer needs to play this game with the following title and tags:
{next_item_title}.
The tags of the game are {next_item_tag}.
The price of the game is {next_item_price}.

You can infer the game's information from the history record, such as price. \
Use ONLY the information mentioned above, especially the history record. \
You're responding to the CUSTOMER. Directly tell the customer the reason. \
Respond in SHORT and CONCISE language. Your response should be WITHIN 75 words.

The output format is as follows:
Game information: <only number of the price>
Recommendation reason: <reason>
"""
    return instruction

if dataset_name == 'ml-100k':
    get_instruction = get_instruction_ml
elif dataset_name == 'Amazon_All_beauty':
    get_instruction = get_instruction_Amazon
elif dataset_name == 'mind_small_dev':
    get_instruction = get_instruction_mind
elif dataset_name == 'steam':
    get_instruction = get_instruction_steam

In [10]:
from tqdm import tqdm
def get_valid_user_list(df):
    valid_user_list = []
    for id in tqdm(df['user_id:token'].unique().tolist()):
        if len(df_inter[df_inter['user_id:token'] == id]) >=5:
            valid_user_list.append(id)
        if len(valid_user_list) == 1000:
            break
    return valid_user_list
valid_user_list = get_valid_user_list(df_inter)
valid_user_list

  0%|          | 9549/2567538 [00:20<1:30:42, 469.97it/s]


[13,
 17,
 31,
 41,
 42,
 75,
 78,
 80,
 85,
 94,
 98,
 100,
 104,
 106,
 109,
 112,
 115,
 122,
 136,
 149,
 154,
 155,
 158,
 159,
 160,
 164,
 171,
 199,
 206,
 225,
 230,
 238,
 240,
 249,
 257,
 263,
 272,
 282,
 283,
 285,
 286,
 295,
 298,
 336,
 337,
 340,
 344,
 347,
 352,
 353,
 360,
 371,
 387,
 390,
 401,
 403,
 414,
 421,
 427,
 428,
 430,
 435,
 440,
 449,
 457,
 461,
 505,
 516,
 521,
 522,
 525,
 527,
 542,
 551,
 552,
 563,
 571,
 575,
 577,
 582,
 586,
 611,
 642,
 643,
 651,
 656,
 660,
 671,
 681,
 682,
 691,
 692,
 693,
 696,
 697,
 700,
 702,
 712,
 737,
 742,
 756,
 758,
 759,
 769,
 783,
 788,
 789,
 794,
 805,
 808,
 815,
 817,
 823,
 832,
 851,
 858,
 860,
 865,
 875,
 910,
 914,
 920,
 921,
 929,
 939,
 940,
 992,
 996,
 1015,
 1018,
 1028,
 1032,
 1054,
 1081,
 1092,
 1100,
 1108,
 1110,
 1113,
 1152,
 1160,
 1171,
 1180,
 1197,
 1201,
 1206,
 1214,
 1223,
 1226,
 1227,
 1232,
 1235,
 1249,
 1283,
 1286,
 1291,
 1294,
 1304,
 1334,
 1335,
 1336,
 1341,
 1357

In [11]:
instruction, response = evaluate(get_instruction(str(9483)))
print(response)

Game information: 29.99
Recommendation reason: As a fan of RPGs, you'll enjoy Fallout 4's immersive storyline and character customization. With its vast open world and engaging combat, this game offers endless hours of entertainment.


In [21]:
response.split('Recommendation reason:')[0].split('Game information:')[-1].strip(' \n')

'29.99'

In [12]:
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore")

random.shuffle(valid_user_list)

result_save_path = f'./results/LLaMA2/{dataset_name}/'
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

count = 0

pbar = tqdm(valid_user_list[:100])
len = 0
for id in pbar:
    len += 1
    df_inter_user = df_inter[df_inter['user_id:token'] == int(id)]
    his_id_list = df_inter_user['product_id:token'].tolist()[-10:]
    uid_series = dataset.token2id(dataset.uid_field, [str(id)])
    topk_score, topk_iid_list = full_sort_topk(uid_series, rec_model, test_data, k=1, device=config['device'])
    rec_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu()) 
    # next_item_info
    next_item = df_item[df_item["id:token"] == int(rec_list)]
    next_item_title = next_item["app_name:token"].values[0]
    next_item_tag = next_item["genres:token_seq"].values[0]
    next_item_price = next_item["price:float"].values[0]
    if next_item_price == 'Free to Play':
        next_item_price = 0

    try:
        instruction, response = evaluate(get_instruction_steam(str(id)))
        price = response.split('Recommendation reason:')[0].split('Game information:')[-1].strip(' \n')
        print(price)
        print(next_item_price)
        if price == next_item_price:
            count += 1
        pbar.set_postfix({'price_acc': count / len})
    except:
        continue

  0%|          | 0/100 [00:00<?, ?it/s]

19.99
19.99


  1%|          | 1/100 [00:04<06:44,  4.08s/it, price_acc=1]

19.99
19.99


  2%|▏         | 2/100 [00:06<04:54,  3.01s/it, price_acc=1]

...

 93%|█████████▎| 93/100 [03:25<00:13,  2.00s/it, price_acc=0.269]

19.99
19.99


 94%|█████████▍| 94/100 [03:27<00:12,  2.12s/it, price_acc=0.277]

19.99
19.99


 95%|█████████▌| 95/100 [03:30<00:11,  2.31s/it, price_acc=0.284]

In [15]:
data = {"instruction": "--------------------Instruction--------------------\nThe history films watched by the customer are:\n1: To Gillian on Her 37th Birthday\nThe class of the movie is Drama Romance.\n2: Eddie\nThe class of the movie is Comedy.\n3: 101 Dalmatians\nThe class of the movie is Children's Comedy.\n4: Broken Arrow\nThe class of the movie is Action Thriller.\n5: Michael\nThe class of the movie is Comedy Romance.\n6: Happy Gilmore\nThe class of the movie is Comedy.\n7: Escape from L.A.\nThe class of the movie is Action Adventure Sci-Fi Thriller.\n8: Grumpier Old Men\nThe class of the movie is Comedy Romance.\n9: Dragonheart\nThe class of the movie is Action Adventure Fantasy.\n10: Fan, The\nThe class of the movie is Thriller.\n\nThe age of the customer is 15, the gender is male and the customer's occupation is student. As a recommender system in the movie domain, based on the customer's historical viewing records, historical viewing movie information and user information, tell the customer why he or she needs to watch this movie, and what are the advantages of this movie.\nGive reasons why the customer needs to watch this movie with the following title and class:\nContact.\nThe class of the movie is Drama Sci-Fi.\n\nUse ONLY the information mentioned above, especially the history record. You're responding to the CUSTOMER. Directly tell the customer the reason. Respond in SHORT and CONCISE language. Your response should be WITHIN 75 words.\n\n\n--------------------Explanation 1--------------------\nBased on your viewing history, I recommend \"Contact\" because it's a thought-provoking drama sci-fi movie that explores themes of human connection and the unknown. As a student, you may find the intellectual and emotional depth of the film engaging and enjoyable. Additionally, the movie's unique blend of drama and sci-fi elements will provide a fresh and exciting viewing experience.\n--------------------Explanation 2--------------------\nDear Customer, you need to watch the movie \"Contact\" because it falls under the category of Drama Sci-Fi. Based on your viewing history, you have watched a variety of genres, including drama and sci-fi. \"Contact\" offers a unique blend of these genres, providing you with an engaging and thought-provoking experience. Additionally, the movie explores the themes of human connection and the possibilities of extraterrestrial life, which align with your interests in romance and adventure. Enjoy the film!\n-----------------------------------------------------\n", "input": "", "output": "1"}
print(data['instruction'])

--------------------Instruction--------------------
The history films watched by the customer are:
1: To Gillian on Her 37th Birthday
The class of the movie is Drama Romance.
2: Eddie
The class of the movie is Comedy.
3: 101 Dalmatians
The class of the movie is Children's Comedy.
4: Broken Arrow
The class of the movie is Action Thriller.
5: Michael
The class of the movie is Comedy Romance.
6: Happy Gilmore
The class of the movie is Comedy.
7: Escape from L.A.
The class of the movie is Action Adventure Sci-Fi Thriller.
8: Grumpier Old Men
The class of the movie is Comedy Romance.
9: Dragonheart
The class of the movie is Action Adventure Fantasy.
10: Fan, The
The class of the movie is Thriller.

The age of the customer is 15, the gender is male and the customer's occupation is student. As a recommender system in the movie domain, based on the customer's historical viewing records, historical viewing movie information and user information, tell the customer why he or she needs to watch th